# Translate docs into other languages

In [1]:
# !pip uninstall googletrans -y
# !pip install googletrans==4.0.0rc1

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Successfully uninstalled googletrans-4.0.0rc1
Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.0rc1-py3-none-any.whl

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
from googletrans import Translator

import os
from os import walk
from os import listdir
from os.path import isfile, join
import time

import openai



openai.api_key = ''



def send_message(prompt, message):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt + message,
            },
        ],
    )
    return completion.choices[0].message.content




In [11]:
send_message(prompt='can you please translate below text into English?', 
             message='我是你爸爸')

'"I am your father."'

In [3]:
translator = Translator()
translated = translator.translate('Mitä sinä teet')
print(translated.text)

What are you doing


In [2]:
Folder_list = ['../Dessert','../Main_Course']

Post_list = []

In [3]:
for folder in Folder_list:
    for root, directories, files in os.walk(folder, topdown=False):
        for name in files:
            print(os.path.join(root, name))
            if '.md' in name:
                Post_list.append(os.path.join(root, name))

../Dessert/Hardware/2021-06-03-Mac-mini.md
../Dessert/Hardware/2021-01-02-Apple-Watch-S5.md
../Dessert/Hardware/2020-04-05-iPad-Pro.md
../Dessert/Hardware/2020-03-14-MX-Keys.md
../Dessert/Tactics/2021-11-17-Articles-Review.md
../Dessert/Tactics/2023-03-26-US-Svc.md
../Dessert/Tactics/2023-03-05-US-DL.md
../Dessert/Tactics/2023-03-04-US-enterance.md
../Dessert/Tactics/2021-07-02-Posting-Workflow.md
../Dessert/Apps/2021-07-12-Paste.md
../Dessert/Apps/2021-10-23-PicGo.md
../Dessert/Apps/2022-05-28-SageMaker-Studio-Lab.md
../Dessert/Apps/2021-06-06-Notion.md
../Dessert/Apps/2021-06-15-Hotapps.md
../Dessert/Apps/2021-10-29-iOS-Shortcuts.md
../Dessert/Apps/2021-12-25-Apache.md
../Dessert/Apps/2021-11-26-APIintro.md
../Dessert/Apps/2022-01-04-Z-Library.md
../Dessert/Apps/2021-07-19-Copilot.md
../Main_Course/Data_Analytics/2021-08-20-practical-recursion-for-data-cleaning.md
../Main_Course/Data_Analytics/2021-06-08-Taobao-Auto-Order-Script.md
../Main_Course/Data_Analytics/2021-07-18-PayPal-Job-

In [4]:
len(Post_list)

99

## Translate files

In [5]:
def translate_post(text_lines, translator, lang):
    split_point_ = 0
    # Find the position that starts the body, the headers and body should be processed separately
    for no_, block_ in enumerate(text_lines):
        if '##' in block_:
            split_point_ = no_
            print(f'Header ends at no.{split_point_} block...')
            break
    
    
    headers_ = text_lines[:split_point_]
    body_ = text_lines[split_point_:]
    
    def translate_header_(_headers_, lang):
        if lang == 'en':
            en_headers_ = _headers_.replace('作者', 'Author:')\
                                    .replace('发布于', 'Posted on ')\
                                    .replace('阅读时间：约','Reading time: ')\
                                    .replace('分钟', ' mins')\
                                    .replace('微信公众号文章链接','WeChat Post Link:')
            return en_headers_
        else:
            print(f'{lang} cannot be well verified, so the original headers are returned.')
            return _headers_
    
    def translate_body_(body_: list, lang: str):
        import time
        import re
        en_body_ = []
        if lang == 'en':
            for text_ in body_:
                # exclusion of special chars
                if 'img' in text_ or text_ == '\n':
                    en_body_.append(text_)
                
                # Skip hyper links
                elif re.findall(r'(\[.+\]\(.+\))', text_):
                    
                    hyper_link_ = re.findall(r'(\[.+\]\(.+\))', text_)

                    if hyper_link_:

                        split_text = re.split(r'(\[.+\]\(.+\))', text_, maxsplit=0)
                        for txt_ in split_text:
                            
                            if txt_ != hyper_link_ and txt_ != '' and  txt_ != '\n':
                                try:
                                    en_body_.append(translator.translate(str(txt_)).text)
                                except:
                                    print(txt_)
                                    pass
                            
                            elif  txt_ != '' and  txt_ != '\n':
                                en_body_.append(txt_)
                            else:
                                pass
                else:
                    # Sleep 300 milliseconds
                    time.sleep(300/1000)
                    en_body_.append(translator.translate(str(text_)).text)
            return en_body_
        else:
            print(f'{lang} cannot be well verified, so the original body are returned.')
            return body_
    
    translated_headers = translate_header_(''.join(headers_), lang)
    translated_body = translate_body_(body_, lang)
    
    return translated_headers + '\n'.join(translated_body)
    

In [13]:

prompt = "translate below text into English: "

chn_posts = {}
en_posts = {}
for post_path in Post_list:
    print(f'Processing {post_path}...')
    with open(post_path, 'r') as f:
        post_text_ = f.readlines()
        if 'Vincent' in ''.join(post_text_):

            chn_posts[post_path] = post_text_
            # split the text if the number of tokens are greater than 4097
            en_posts[post_path] = send_message(prompt=prompt,
                                                    message=''.join(post_text_)
                                                    ) 
            time.sleep(300/1000)



Processing ../Dessert/Hardware/2021-06-03-Mac-mini.md...
../Dessert/Hardware/2021-06-03-Mac-mini.md
Processing ../Dessert/Hardware/2021-01-02-Apple-Watch-S5.md...
../Dessert/Hardware/2021-01-02-Apple-Watch-S5.md
Processing ../Dessert/Hardware/2020-04-05-iPad-Pro.md...
../Dessert/Hardware/2020-04-05-iPad-Pro.md


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 5691 tokens. Please reduce the length of the messages.

In [7]:
en_posts_copy = en_posts.copy()

In [8]:
len(en_posts_copy)

0

In [14]:
en_posts_new = {}
for path, text in en_posts_copy.items():
    en_path = path.replace('../','../en/')
    en_posts_new[en_path] = text

In [18]:
test_key = list(en_posts_copy.keys())[0]
print(en_posts_copy[test_key])

---
template: overrides/blogs.html
tags:
  - hardware
---

# M1 Mac mini 使用半年体验 - Mac的新未来

!!! info
    Author:：[Vincent](https://github.com/Realvincentyuan)，Posted on 2021-06-03，Reading time: 12 mins，WeChat Post Link:：[:fontawesome-solid-link:](https://mp.weixin.qq.com/s?__biz=MzI4Mjk3NzgxOQ==&mid=2247483932&idx=1&sn=533ebbb843dac745675da8657ffeafdc&chksm=eb90f168dce7787ef270db462c593e89b0f57fbeadd865b90aead97ff48c3338bb6fbffa0471&token=2120399287&lang=zh_CN#rd)


<figure>
  <img src="https://user-images.githubusercontent.com/26101303/120489285-1959a100-c3ea-11eb-8025-f96511845c1d.jpg" width="600" />
  <figcaption>M1 Mac mini</figcaption>
</figure>


## 1 Personal use scenario


I have been using MAC for 6 years. Except for Mac Pro, the remaining series of models have been the main workplace, so it can be regarded as a slightly experienced user.The machine on hand now is M1 Mac Mini (8G memory + 256G SSD), which was published on November 11, 2020, and purchased on December 18, 2020.As

## Write the EN files

In [19]:
from pathlib import Path

for en_path_, text_ in en_posts.items():
    filename = Path(en_path_).parent
    filename.mkdir(parents=True, exist_ok=True)
    filename.touch(exist_ok=True)
    with open(en_path_, 'w+') as f:


        f.writelines(text_)

In [1]:
!pip install -q openai

In [16]:
cn_text = """
translate below text into English

---
template: overrides/blogs.html
tags:
  - python
---

# Python元组常用操作小技巧

!!! info
    作者：[Vincent](https://github.com/Realvincentyuan)，发布于2021-08-07，阅读时间：约6分钟，微信公众号文章链接：[:fontawesome-solid-link:](https://mp.weixin.qq.com/s/GmZ1Du4qJRai7D7oXKh75w)

## 1 前言

Python作为当下数据科学、人工智能领域炙手可热的编程语言受到了非常多的关注，有很多人都在学习。但是在追求卓越的路上，一定不要忽视了基础，比如常用数据结构、语法规范、编程思维的最佳实践，对这些最基础的事情了如指掌，在这基础之上的工作也会游刃有余。

我们先来回顾和总结Python数据结构里常用操作：Python中常见的数据结构可以统称为容器（container）。序列（如列表和元组）、映射（如字典）以及集合（set）是三类主要的容器。而扁平序列如str、bytes、bytearray、memoryview 和 array.array等不在这篇文章的讨论范围内。

在此，我们先从元组开始说起。

## 2 元组不仅是不可变的列表

### 2.1 元组的记录作用

元组区别于列表的显著特征之一就是它不能被修改，但其另外一个作用就是[用于没有字段名的记录](https://book.douban.com/subject/27028517/)。因为后者经常被忽略，我们先来看看元组作为记录的作用。

使用括号就可以定义一个元组。元组中的每个元素都存放了记录中一个字段的数据，外加这个字段的位置。正是这个位置信息给数据赋予了意义。下面的例子中，元组就被当作记录加以利用：

```Python
# 洛杉矶国际机场的经纬度，用元组记录
lax_coordinates = (33.9425, -118.408056)

# 东京市的一些信息：市名、年份、人口（单位：百万）、人口变化 （单位：百分比）和面积（单位：平方千米），用元组记录
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)

#一个元组列表，元组的形式为 (country_code, passport_number)。
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'),('ESP', 'XDA205856')]

#在迭代的过程中，passport 变量被绑定到每个元组上。% 格式运算符能被匹配到对应的元组元素上。
for passport in sorted(traveler_ids):
  print('%s/%s' % passport)
```

输出为：

```
BRA/CE342567
ESP/XDA205856
USA/31195855
```

### 2.2 元组的拆包

上述for循环中的操作提取了元组中的元素，也叫作拆包（unpacking）。平行赋值是对元组拆包很好的应用，示例如下：

```Python
# 洛杉矶国际机场的经纬度，用元组记录
lax_coordinates = (33.9425, -118.408056)
# 拆包元组，将元组中的元素分别赋值给对应的变量
latitude, longitude = lax_coordinates
```

还有一个经典而优雅的应用是交换变量的值：

```Python
a, b = b, a
```

### 2.3 元组作为函数参数

用`*`运算符把一个可迭代对象拆开作为函数的参数，例如Python的内置函数divmod接收两个数字类型的参数，返回商和余数。以下范例将使用`*`将元组传入函数。

```Python
t = (20, 8)
quotient, remainder = divmod(*t)
print(quotient, remainder)
```
输出为：

```
2 4
```

### 2.4 元组作为函数返回值

有些函数有多个返回值，将其赋给一个变量时，变量类型即是元组：

```Python
import numpy as np
arr = np.random.randint(10, size=8)

def count_sum(arr):
   count = len(arr)
   sum = arr.sum()
   return count, sum

result = count_sum(arr)
print(result)
```

输出为：

```
(8, 40)
```

### 2.5 zip函数

zip是Python的内置函数，能够接收两个或多个序列，并组成一个元组列表，在Python3中会返回一个迭代器，如下所示：

```Python
list_a = [1,2,3]
list_b = ['a','b','c']

for i in zip(list_a, list_b):
    print(i)
```

输出为：

```
(1, 'a')
(2, 'b')
(3, 'c')
```

### 2.6 其他常规操作

元组当然也支持一些常规操作，如对于元组`a = (1, 'y', 5, 5, 'x')`：

- 索引，如`a.index('x')`
- 切片，如`a[1:4]`
- 计数，如`a.count(5)`
- 排序，如`a.sort()`
- 合并元组，如`c = a + a`

## 3 总结

上述内容不仅涵盖了元组的基本操作，同时也结合了实际工作中常搭配使用的其他函数、运算符等。在回顾这些知识时主要参考了两本经典的Python编程书籍：《流畅的Python》和《像计算机科学家一样思考Python》，有兴趣的朋友可以深入阅读！

希望这篇文章对你有帮助，下回将总结Python列表的使用技巧。

<figure>
  <img src="https://cdn.jsdelivr.net/gh/BulletTech2021/Pics/2021-6-14/1623639526512-1080P%20(Full%20HD)%20-%20Tail%20Pic.png" width="500" />
</figure>



"""

send_message(message=cn_text)

'# Python Tuple Tips and Tricks\n\n!!! info\n    Author: [Vincent](https://github.com/Realvincentyuan), published on August 7, 2021, Read time: about 6 minutes, WeChat Official Account article link: [:fontawesome-solid-link:](https://mp.weixin.qq.com/s/GmZ1Du4qJRai7D7oXKh75w)\n\n## 1 Introduction\n\nPython, as a popular programming language in the fields of data science and artificial intelligence, has received a lot of attention. Many people are learning it. However, in the pursuit of excellence, we must not neglect the basics, such as the best practices for common data structures, syntax rules, and programming thinking. If we are familiar with these basics, we can work more effectively on top of them. \n\nLet\'s start by reviewing and summarizing the common operations in Python data structures: Common data structures in Python can be collectively referred to as containers. Sequences (such as lists and tuples), mappings (such as dictionaries), and sets are the three main types of cont

In [18]:
en_text = """
# Python Tuple Tips and Tricks\n\n!!! info\n    Author: [Vincent](https://github.com/Realvincentyuan), published on August 7, 2021, Read time: about 6 minutes, WeChat Official Account article link: [:fontawesome-solid-link:](https://mp.weixin.qq.com/s/GmZ1Du4qJRai7D7oXKh75w)\n\n## 1 Introduction\n\nPython, as a popular programming language in the fields of data science and artificial intelligence, has received a lot of attention. Many people are learning it. However, in the pursuit of excellence, we must not neglect the basics, such as the best practices for common data structures, syntax rules, and programming thinking. If we are familiar with these basics, we can work more effectively on top of them. \n\nLet\'s start by reviewing and summarizing the common operations in Python data structures: Common data structures in Python can be collectively referred to as containers. Sequences (such as lists and tuples), mappings (such as dictionaries), and sets are the three main types of containers. Flat sequences such as str, bytes, bytearray, memoryview, and array.array are not within the scope of this article.\n\nNow, let\'s start with tuples.\n\n## 2 Tuples are more than just immutable lists\n\n### 2.1 Record role of tuples\n\nOne of the significant features of tuples compared to lists is that they cannot be modified, but another role of tuples is that they are used for [record without field names](https://book.douban.com/subject/27028517/). Since the latter is often overlooked, let\'s take a look at the role of tuples as records first.\n\nBy placing elements in parentheses, you can define a tuple. Each element in the tuple stores data for one field in the record, plus the position of that field. It is this positional information that gives meaning to the data. In the following example, the tuple is used as a record:\n\n```Python\n# The latitude and longitude of Los Angeles International Airport in tuple format\nlax_coordinates = (33.9425, -118.408056)\n\n# Tokyo City\'s information: name, year, population (in millions), population change (in percentage), and area (in square km) in tuple format\ncity, year, pop, chg, area = (\'Tokyo\', 2003, 32450, 0.66, 8014)\n\n# A list of tuples, the form of the tuple is (country_code, passport_number).\ntraveler_ids = [(\'USA\', \'31195855\'), (\'BRA\', \'CE342567\'),(\'ESP\', \'XDA205856\')]\n\n# In the process of iteration, the passport variable is bound to each tuple. The % format operator can be matched to the corresponding tuple element.\nfor passport in sorted(traveler_ids):\n  print(\'%s/%s\' % passport)\n```\n\nThe output is:\n```\nBRA/CE342567\nESP/XDA205856\nUSA/31195855\n```\n\n### 2.2 Tuple unpacking\n\nThe operation in the above for loop extracts elements from the tuple, which is also called unpacking. Parallel assignment is a good use of tuple unpacking, as shown in the following example:\n\n```Python\n# The latitude and longitude of Los Angeles International Airport in tuple format\nlax_coordinates = (33.9425, -118.408056)\n# Unpack the tuple and assign each element in the tuple to the corresponding variable\nlatitude, longitude = lax_coordinates\n```\n\nAnother classic and elegant application is to swap the values of variables:\n\n```Python\na, b = b, a\n```\n\n### 2.3 Tuple as a function parameter\n\nUse the `*` operator to unpack an iterable object as a function parameter. For example, the built-in Python function `divmod` takes two numeric parameters and returns the quotient and remainder. The following example uses `*` to pass a tuple into the function.\n\n```Python\nt = (20, 8)\nquotient, remainder = divmod(*t)\nprint(quotient, remainder)\n```\nThe output is:\n```\n2 4\n```\n\n### 2.4 Tuple as a function return value\n\nSome functions have multiple return values. When they are assigned to a variable, the variable type is a tuple:\n\n```Python\nimport numpy as np\narr = np.random.randint(10, size=8)\n\ndef count_sum(arr):\n   count = len(arr)\n   sum = arr.sum()\n   return count, sum\n\nresult = count_sum(arr)\nprint(result)\n```\n\nThe output is:\n```\n(8, 40)\n```\n\n### 2.5 zip function\n\n`zip` is a built-in function in Python that can take two or more sequences and combine them into a list of tuples. In Python 3, it returns an iterator as shown below:\n\n```Python\nlist_a = [1,2,3]\nlist_b = [\'a\',\'b\',\'c\']\n\nfor i in zip(list_a, list_b):\n    print(i)\n```\n\nThe output is:\n```\n(1, \'a\')\n(2, \'b\')\n(3, \'c\')\n```\n\n### 2.6 Other common operations\n\nOf course, tuples also support some common operations. For example, for tuple `a = (1, \'y\', 5, 5, \'x\')`:\n\n- Indexing, such as `a.index(\'x\')`\n- Slicing, such as `a[1:4]`\n- Counting, such as `a.count(5)`\n- Sorting, such as `a.sort()`\n- Merging tuples, such as `c = a + a`\n\n## 3 Summary\n\nThe above content not only covers the basic operations of tuples but also combines other functions, operators, and so on commonly used in practical work. When reviewing this knowledge, I mainly referenced two classic Python programming books: "Fluent Python" and "Think Like a Computer Scientist: Learning with Python 3." Interested friends can read them in detail!\n\nI hope this article will be helpful to you, and next time we will summarize the tips for using Python lists. \n\n<figure>\n  <img src="https://cdn.jsdelivr.net/gh/BulletTech2021/Pics/2021-6-14/1623639526512-1080P%20(Full%20HD)%20-%20Tail%20Pic.png" width="500" />\n</figure>'
"""

print(en_text)


# Python Tuple Tips and Tricks

!!! info
    Author: [Vincent](https://github.com/Realvincentyuan), published on August 7, 2021, Read time: about 6 minutes, WeChat Official Account article link: [:fontawesome-solid-link:](https://mp.weixin.qq.com/s/GmZ1Du4qJRai7D7oXKh75w)

## 1 Introduction

Python, as a popular programming language in the fields of data science and artificial intelligence, has received a lot of attention. Many people are learning it. However, in the pursuit of excellence, we must not neglect the basics, such as the best practices for common data structures, syntax rules, and programming thinking. If we are familiar with these basics, we can work more effectively on top of them. 

Let's start by reviewing and summarizing the common operations in Python data structures: Common data structures in Python can be collectively referred to as containers. Sequences (such as lists and tuples), mappings (such as dictionaries), and sets are the three main types of containers. Fl